In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
#Read dataset
df=pd.read_csv("../input/car_ad.csv", encoding='latin-1')

In [ ]:
df = df.drop(df[df.price <= 0 ].index)

In [ ]:
df.price[df.price ==0].count()

In [ ]:
df.head(3)

In [ ]:
#df = df.price[df.price > 0]
#Drop column where price is zero

df = df.drop(df[df.price == 0].index)

In [ ]:
df.price[df.price == 0].count()

In [ ]:
df.shape

In [ ]:
#visualize dataset
sns.countplot(df['body'])

In [ ]:
sns.boxplot(x='mileage',y='price',data=df)

In [ ]:
sns.regplot(x='mileage',y='price',data=df)

In [ ]:
sns.countplot(df['registration'])

In [ ]:
sns.boxplot(x='registration',y='price',data=df)

In [ ]:
sns.regplot(x='engV',y='price',data=df)

In [ ]:
sns.countplot(df['engType'])

In [ ]:

#hot encode
from sklearn import model_selection, preprocessing
for c in df.columns:
    if df[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(df[c].values)) 
        df[c] = lbl.transform(list(df[c].values))
        #x_train.drop(c,axis=1,inplace=True)

In [ ]:
df.head(3)

In [ ]:
y_train = df["price"]
x_train = df.drop(["price"], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
data_train, data_test, label_train, label_test = train_test_split(x_train, y_train, test_size = 0.2, random_state = 42)

In [ ]:
x_train.head(3)

In [ ]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

In [ ]:
df.price.skew()

In [ ]:
#with log transform
label_test_log=np.log(label_test)

In [ ]:
label_train_log=np.log(label_train)

In [ ]:
dtrain_log = xgb.DMatrix(data_train, label_train_log)

In [ ]:
label_train1=np.log(label_train)

In [ ]:
#with log
label_train_log.skew()

In [ ]:
dtrain = xgb.DMatrix(data_train, label_train)

In [ ]:
#without log transform
cv_output = xgb.cv(xgb_params, dtrain, num_boost_round=1000, early_stopping_rounds=20,
    verbose_eval=50, show_stdv=False)

In [ ]:
#with log transform
cv_output_log = xgb.cv(xgb_params, dtrain_log, num_boost_round=1000, early_stopping_rounds=20,
    verbose_eval=50, show_stdv=False)

In [ ]:
num_boost_rounds = len(cv_output)
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round= 

num_boost_rounds)

In [ ]:
#with log transform
num_boost_rounds = len(cv_output)
model_log = xgb.train(dict(xgb_params, silent=0), dtrain_log, num_boost_round= 

num_boost_rounds)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 13))
xgb.plot_importance(model, max_num_features=50, height=0.5, ax=ax)

In [ ]:
#without log transformation
dtest=xgb.DMatrix(data_test)

In [ ]:
data_test.head(2)

In [ ]:
#without log
y_predict = model.predict(dtest)
out = pd.DataFrame({'Actual_price': label_test, 'predict_price': y_predict,'Diff' :(label_test-y_predict)})
out[['Actual_price','predict_price','Diff']].head(5)

In [ ]:
#with log transformation
y_predict_log = model_log.predict(dtest)
y_predict_log=np.exp(y_predict_log)
out_log = pd.DataFrame({'Actual_price': label_test, 'predict_price': y_predict_log,'Diff' :(label_test-y_predict_log)})
out_log[['Actual_price','predict_price','Diff']].head(5)

In [ ]:
#log transformation
sns.regplot(out_log['predict_price'],out_log['Diff'])

In [ ]:
out[['Actual_price','predict_price','Diff']].head(3)

In [ ]:
sns.regplot(out['Actual_price'],out['Diff'])

In [ ]:
sns.regplot(out['predict_price'],out['Diff'])